In [25]:
import pandas as pd

df=pd.read_csv("cleaned_file.csv")


In [26]:
import pandas as pd

# Load the CSV file
file_path = "cleaned_file.csv"  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm data loading
print("Sample data from CSV:")
print(df.head())

# Check column names to ensure they match the expected ones
print("\nColumn names:", df.columns)

# If column names need to be standardized
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# Display questions and answers
print("\nQuestions and Answers:")
for index, row in df.iterrows():
    print(f"Q: {row['question']}")
    print(f"A: {row['answer']}\n")


Sample data from CSV:
                        question  \
0               What is the fees   
1  What is the cost of education   
2         What is the curriculum   
3            How are the faculty   
4     How many faculty are there   

                                              answer  
0  Sitare University is committed to providing hi...  
1  Sitare University is committed to providing hi...  
2  The curriculum at Sitare University focuses on...  
3  Sitare University has very high standards for ...  
4  As of 2024 we have more than 8 full time facul...  

Column names: Index(['question', 'answer'], dtype='object')

Questions and Answers:
Q: What is the fees
A: Sitare University is committed to providing highquality Computer Science education to bright students from underprivileged backgrounds We offer a 100 scholarship to students who meet our criteria securing a minimum of 90 percentile in JEE Mains and having a family income of less than 3 lakhs per annum Our goal is to empow

In [27]:
df2=df.drop_duplicates()

In [28]:
df2

,question,answer
0,What is the fees,Sitare University is committed to providing hi...
1,What is the cost of education,Sitare University is committed to providing hi...
2,What is the curriculum,The curriculum at Sitare University focuses on...
3,How are the faculty,Sitare University has very high standards for ...
4,How many faculty are there,As of 2024 we have more than 8 full time facul...
...,...,...
245,Is it better than nits and iiits,We strongly believe that Sitare University is ...
246,Why Sitare is best over private collages,We strongly believe that Sitare University is ...
247,How many clubs are there,There are total 7 clubs as follows Coding club...
248,Which clubs are there,There are total 7 clubs as follows Coding club...


In [29]:
print("Missing values:\n", df2.isnull().sum())

Missing values:
 question    0
answer      0
dtype: int64


In [24]:
import pandas as pd
import re
from nltk import download

# Download necessary NLTK data (if required in the future for advanced processing)
download('punkt')

# Function to clean text
def clean_text(text):
    # Convert to string (to handle non-string entries)
    text = str(text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text.strip())
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Load the CSV file
file_path = 'chatbot-combined (1).csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Clean both the 'question' and 'answer' columns
columns_to_clean = ['question', 'answer']
for column in columns_to_clean:
    if column in df.columns:
        df[column] = df[column].apply(clean_text)
    else:
        print(f"Warning: Column '{column}' not found in the CSV!")

# Save the cleaned CSV file
output_path = 'cleaned_file.csv'  # Replace with your desired output file path
df.to_csv(output_path, index=False)

print(f"Cleaned CSV saved at {output_path}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Cleaned CSV saved at cleaned_file.csv


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine or focus on 'answer' column for vectorization
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'answer' column
tfidf_matrix = tfidf_vectorizer.fit_transform(df2['answer'])

# Print the shape of the TF-IDF matrix
print("\nTF-IDF Matrix shape:", tfidf_matrix.shape)

# Optionally, inspect the vocabulary
print("\nTF-IDF Vocabulary:", tfidf_vectorizer.vocabulary_)



TF-IDF Matrix shape: (249, 750)

TF-IDF Vocabulary: {'sitare': 592, 'university': 696, 'is': 369, 'committed': 148, 'to': 673, 'providing': 528, 'highquality': 319, 'computer': 156, 'science': 568, 'education': 215, 'bright': 117, 'students': 627, 'from': 278, 'underprivileged': 692, 'backgrounds': 86, 'we': 719, 'offer': 461, '100': 1, 'scholarship': 567, 'who': 731, 'meet': 422, 'our': 479, 'criteria': 171, 'securing': 575, 'minimum': 430, 'of': 460, '90': 23, 'percentile': 495, 'in': 343, 'jee': 375, 'mains': 407, 'and': 58, 'having': 310, 'family': 245, 'income': 346, 'less': 393, 'than': 653, 'lakhs': 384, 'per': 494, 'annum': 60, 'goal': 298, 'empower': 220, 'talented': 645, 'with': 734, 'exceptional': 236, 'opportunities': 473, 'ensuring': 226, 'that': 654, 'financial': 251, 'constraints': 161, 'do': 198, 'not': 456, 'hinder': 321, 'their': 656, 'potential': 512, 'the': 655, 'curriculum': 174, 'at': 78, 'focuses': 257, 'on': 467, 'industry': 352, 'oriented': 477, 'cs': 172, 'le

In [31]:
# Load Sentence-Transformers Model (ensure you do this only once)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def generate_embedding(text, model):
    """
    Generate embedding for a given text using the provided model.
    """
    return model.encode(text).tolist()  # Convert to list for PostgreSQL insertion


In [35]:
import psycopg2
from sentence_transformers import SentenceTransformer
import pandas as pd  # Ensure this is installed if using DataFrame (df2)

# Connection parameters
host = "localhost"
user = "postgres"
password = "SQLanshu728"
port = "5432"
database = "postgres"  # Replace with your database name

try:
    # Step 1: Connect to PostgreSQL and setup pgvector
    connection = psycopg2.connect(
        host=host,
        user=user,
        password=password,
        port=port,
        database=database
    )
    connection.autocommit = True
    cursor = connection.cursor()

    # Enable pgvector extension
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("pgvector extension is ready.")

    # Create or modify the table for storing question embeddings
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS qa1_embeddings (
            id SERIAL PRIMARY KEY,
            question TEXT,
            answer TEXT,  -- New column for answers
            embedding vector(384)  -- 384 dimensions for all-MiniLM-L6-v2 model
        );
    """)
    print("Table 'qa1_embeddings' with 'answer' column is ready.")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

# Step 2: Load Sentence-Transformers Model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to generate embeddings
def generate_embedding(text, model):
    return model.encode(text).tolist()  # Convert to list for PostgreSQL insertion

# Example DataFrame (df2) - Replace this with your actual data source
# Ensure your data has 'question' and 'answer' columns
data =df2


# Step 3: Insert Data into PostgreSQL
try:
    for index, row in df2.iterrows():  # Ensure df2 is defined and has 'question' and 'answer' columns
        question = row["question"]
        answer = row["answer"]

        # Generate embedding for the question
        embedding = generate_embedding(question, model)

        # Insert into PostgreSQL
        cursor.execute("""
            INSERT INTO qa1_embeddings (question, answer, embedding)
            VALUES (%s, %s, %s);
        """, (question, answer, embedding))
        print(f"Inserted row {index + 1}: {question}")

except Exception as e:
    print("Error while processing data:", e)

# Close the connection
if 'connection' in locals() and connection:
    cursor.close()
    connection.close()
    print("\nPostgreSQL connection is closed.")


pgvector extension is ready.
Table 'qa1_embeddings' with 'answer' column is ready.
Inserted row 1: What is the fees
Inserted row 2: What is the cost of education
Inserted row 3: What is the curriculum
Inserted row 4: How are the faculty
Inserted row 5: How many faculty are there
Inserted row 6: How many permanent faculty are there
Inserted row 7: Name all the faculty name with there subjects
Inserted row 8: How many visiting faculty are there
Inserted row 9: Do you have faculty from abroad
Inserted row 10: What are the qualifications of your faculty
Inserted row 11: How many students are there
Inserted row 12: When did the university start
Inserted row 13: Where is the university located
Inserted row 14: When will the campus be complete
Inserted row 15: When will you move to Indore
Inserted row 16: When will you move to Bhopal
Inserted row 17: When will you move to the permanent campus
Inserted row 18: Who is the founder
Inserted row 19: Who started Sitare University
Inserted row 20: W

it will connect with database you need to just enter your details instead of what have been given.Aloso it will make imbedding and insert in table named qa1_embedding.

In [36]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text
import pandas as pd

class CollegeChatbot:
    def __init__(self, db_params):
        # Initialize the sentence transformer model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Connect to the database
        try:
            self.engine = create_engine(
                f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"
            )
            with self.engine.connect() as connection:
                connection.execute(text("SELECT 1"))
            print("Database connection successful!")
        except Exception as e:
            print(f"Error connecting to the database: {e}")
            raise

    def embed_query(self, query):
        """Generate an embedding for the given query."""
        try:
            embedding = self.model.encode(query)
            return embedding
        except Exception as e:
            print(f"Error generating embedding: {e}")
            raise

    def find_similar_questions(self, query_embedding):
        """Retrieve the most similar questions from the database."""
        try:
            with self.engine.connect() as connection:
                sql_query = text("""
                    SELECT id, question, answer, 
                           (embedding <=> CAST(:query_embedding AS vector)) AS cosine_distance
                    FROM qa1_embeddings
                    ORDER BY cosine_distance
                    LIMIT 5;
                """)
                result = connection.execute(sql_query, {'query_embedding': query_embedding.tolist()})
                return pd.DataFrame(result.fetchall(), columns=['id', 'question', 'answer', 'cosine_distance'])
        except Exception as e:
            print(f"Error finding similar questions: {e}")
            raise

    def get_answer(self, user_query, similarity_threshold=0.4):
        """
        Get the answer to a user's query.
        :param user_query: The user's query as a string.
        :param similarity_threshold: Minimum similarity score for a confident answer.
        :return: A dictionary with the answer and similar questions.
        """
        try:
            # Generate query embedding
            query_embedding = self.embed_query(user_query)
            
            # Retrieve similar questions
            similar_questions = self.find_similar_questions(query_embedding)
            
            if not similar_questions.empty:
                # Get the top match
                top_match = similar_questions.iloc[0]
                similarity_score = 1 - top_match['cosine_distance']
                
                # Return answer and details if above threshold
                if similarity_score >= similarity_threshold:
                    return {
                        'answer': top_match['answer'],
                        'original_question': top_match['question'],
                        'similarity_score': similarity_score,
                        'similar_questions': similar_questions.to_dict('records')
                    }
                else:
                    return {
                        'answer': "Sorry, I couldn't find a confidently matching answer.",
                        'similarity_score': similarity_score,
                        'similar_questions': similar_questions.to_dict('records')
                    }
            else:
                return {'answer': "Sorry, no similar questions were found."}
        except Exception as e:
            print(f"Error getting answer: {e}")
            return {'answer': f"An error occurred: {e}"}

# Setup database connection parameters
db_params = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': "SQLanshu728",
    'port': '5432'
}

# Initialize chatbot instance
chatbot = CollegeChatbot(db_params)


Database connection successful!


In [37]:
user_query = input("Ask your question: ")
result = chatbot.get_answer(user_query)

# Print the question the user asked
print(f"\nYour Question: {user_query}")

# Print the main answer
print(f"\nAnswer: {result['answer']}")

# Print the exact question from the database, if found
if 'original_question' in result:
    print(f"\nOriginal Question in Database: {result['original_question']}")

# Print similar questions, if any
if 'similar_questions' in result and result['similar_questions']:
    print("\nSimilar Questions:")
    for q in result['similar_questions']:
        print(f"- {q['question']} (Distance: {q['cosine_distance']:.4f})")

# Print similarity score
if 'similarity_score' in result:
    print(f"\nSimilarity Score: {result['similarity_score']:.4f}")



Your Question: what is fee?

Answer: Sitare University is committed to providing highquality Computer Science education to bright students from underprivileged backgrounds We offer a 100 scholarship to students who meet our criteria securing a minimum of 90 percentile in JEE Mains and having a family income of less than 3 lakhs per annum Our goal is to empower talented students with exceptional opportunities ensuring that financial constraints do not hinder their potential

Original Question in Database: What is the fees

Similar Questions:
- What is the fees (Distance: 0.1332)
- Is there any application fee (Distance: 0.4385)
- Is there any rule if the student fails to perform than money is charged (Distance: 0.6369)
- What is the cost of education (Distance: 0.7137)
- it is not a scam (Distance: 0.7567)

Similarity Score: 0.8668
